## Using preprocessed annotations for comparisons

This notebook imports manual annotations and compares them with the automated metrics, by individual algorithm. <br>
Their agreement is analyzed by various metrics, including sensitivity, specificity, PPV, NPV, Rand Accuracy. <br>
These are calculated at the bottom of this script. 

CAUTION:
    This method requires the test_code and test month to be present for each gold-standard entry

In [ ]:
# Set up local environment
from cw_package.setup_cw_env import *
from pylab import *
from cw_package import prDF
from au_package import assess_agreement, prep_Au_standard, standardize_flags
import pickle
#import re
jacks_verification

In [ ]:
# name local variables
coded_date = '2017_07_12allfeed_woI10'
earliest_date= '2016_04_01'
time_period= ['2016_4','2016_5','2016_6','2016_7','2016_8','2016_9']
elevenkeys= ['CW_cerv','CW_card','CW_vitd','CW_bph','CW_lbp','CW_feed','CW_psyc','CW_dexa','CW_narc',
             'CW_nonpreop','CW_catpreop']
stdflag_string= 'as_annotated'

GEMdicts = ['10to10','claimto9best', 'claimto9reimb', 'refto10best','refto10gems']

### Import semi-preprocessed annotations data

In [ ]:
# Load annotations from the original annotations done for first CW version
treasurechest_r = pd.read_pickle('./preprocessed/'+coded_date+'/pickled_treasurechest_r_'+coded_date+'.p')
# Load annotations from the Seibert's annotations done for Feb 28th data CW version
treasurechest_s1_r = pd.read_pickle('./preprocessed/'+coded_date+'/pickled_treasurechest_s1_r_'+coded_date+'.p')
# Load annotations from the Seibert's annotations done for May 14 CW version
treasurechest_s2_r = pd.read_pickle('./preprocessed/'+coded_date+'/pickled_treasurechest_s2_r_'+coded_date+'.p')



## Import GEM implementation data

In [ ]:
# Load compiled numerators and denominators from different GEM implementation analyses
numerators_lt   = pd.read_pickle('./preprocessed/'+coded_date+'/pickled_compilednumerators_'+coded_date+'_long.p')
denominators_lt = pd.read_pickle('./preprocessed/'+coded_date+'/pickled_compileddenominators_'+coded_date+'_long.p')

### Removing the numerators from the denominators in each GEM implementation
Convert denominators to specifically "Not-numerator" cases

In [ ]:
# Create list of tuples, that are more appropriately identified as "Not Numerators"
notnumerators_lt = [None]*len(numerators_lt)
for x in range(len(numerators_lt)):
    if denominators_lt[x][0][3:]!=numerators_lt[x][0][3:]:
        raise ValueError('numerators and denominators have different GEMs or order to GEMs')
    else:
        num_MRNonly = numerators_lt[x][1]
        num_MRNonly = num_MRNonly[num_MRNonly.Metric.isin(['CW_bph','CW_narc','CW_feed','CW_psyc','CW_lbp'])]
        num_MRNonly = num_MRNonly[['MRN','TEST_DATE_month','Metric']]
        num_MRNonly['bean']=1
        
        num_3merge = numerators_lt[x][1]
        num_3merge = num_3merge[~num_3merge.Metric.isin(['CW_bph','CW_narc','CW_feed','CW_psyc','CW_lbp'])]
        num_3merge = num_3merge[['MRN','TEST_CODE','TEST_DATE_a','TEST_DATE_month','Metric']]
        num_3merge['bean']=1
        
        
        den_MRNonly = denominators_lt[x][1]
        den_MRNonly = den_MRNonly[den_MRNonly.Metric.isin(['CW_bph','CW_narc','CW_feed','CW_psyc','CW_lbp'])]
        
        den_3merge = denominators_lt[x][1]
        den_3merge = den_3merge[~den_3merge.Metric.isin(['CW_bph','CW_narc','CW_feed','CW_psyc','CW_lbp'])]
        
        den_MRNonly = den_MRNonly.merge(num_MRNonly, on=['MRN','TEST_DATE_month','Metric'], how='left')
        den_3merge  = den_3merge.merge(num_3merge, 
                                       on=['MRN','TEST_CODE','TEST_DATE_a','TEST_DATE_month','Metric'],
                                       how='left')
        
        
    
        notnum = pd.concat([den_MRNonly, den_3merge])
        notnum = notnum[notnum.bean!=1]
        notnumerators_lt[x]= (denominators_lt[x][0][3:], notnum)
    

At this point,
numerators_lt and notnumerators_lt are the analyses' results for assigning numerator status
(rather than numerator and denominator)

In [ ]:
# assign standardized label
for x in notnumerators_lt:
    x[1]['Term per GEM eval']='Not_Numer'
for x in numerators_lt:
    x[1]['Term per GEM eval']='Numer'

# concatenate them into single dataframes by Metric, w/in individual GEM implementations
allterms_lt = [None]*len(numerators_lt)
for x in range(len(numerators_lt)):
    allterms_lt[x]=(numerators_lt[x][0][3:], pd.concat([notnumerators_lt[x][1], numerators_lt[x][1]]))
    allterms_lt[x][1][allterms_lt[x][0]]=1

In [ ]:
avail_lt = pd.concat([x[1].groupby(['Metric','Term per GEM eval']).count() for x in allterms_lt], axis=1)
avail_lt = avail_lt[['_'+x for x in GEMdicts]]
avail_lt

#### Now, work with the gold standards

#### Prep the gold standards by standardizing the flags to match
depends on stdflag_string

In [ ]:
treasurechest_r_std=treasurechest_r
treasurechest_s1_r_std=treasurechest_s1_r
treasurechest_s2_r_std=treasurechest_s2_r


+ Take the numerators from the analyses that are in [treasurechest numerators, not numerators]
+ See how many of them are marked as such in the treasurechest
+ See how many are marked as not numerators

In [ ]:
for x in treasurechest_r_std:
    #print(x['Metric'])
    x['linked']=pd.concat([standardize_flags(x['Numerator'],stdflag_string),standardize_flags(x['Denominator'],stdflag_string)],axis=0)
    # following line makes sure that incident_service counted once if it was in both numerator and denominator
    x['linked_a']=x['linked'].groupby(['MRN','TEST_CODE','Term_assessed','TEST_DATE_month','Gold_Standard']).count()
    x['linked_a']=x['linked_a'].reset_index()
    x['linked_a']['Metric']=x['Metric']
print('Annotation flags have been standardized, per [{}-keyword] setting'.format(stdflag_string))

In [ ]:
for x in treasurechest_s1_r_std:
    print(x['Metric'])
    x['linked']= standardize_flags(x['Annotated'],stdflag_string)
    x['linked_a']=x['linked'].groupby(['MRN','TEST_CODE','Term_assessed','TEST_DATE_month','Gold_Standard']).count()
    x['linked_a']=x['linked_a'].reset_index()
    x['linked_a']['Metric']=x['Metric']
    x['linked_a']=x['linked_a'][x['linked_a'].Term_assessed.isin(['Numer','Not_Numer'])]

In [ ]:
for x in treasurechest_s2_r_std:
    print(x['Metric'])
    x['linked']= standardize_flags(x['Annotated'],stdflag_string)
    x['linked_a']=x['linked'].groupby(['MRN','TEST_CODE','Term_assessed','TEST_DATE_month','Gold_Standard']).count()
    x['linked_a']=x['linked_a'].reset_index()
    x['linked_a']['Metric']=x['Metric']
    x['linked_a']=x['linked_a'][x['linked_a'].Term_assessed.isin(['Numer','Not_Numer'])]

In [ ]:
singlepotofAu_orig= pd.concat([x['linked_a'] for x in treasurechest_r_std])
singlepotofAu_s1 = pd.concat([x['linked_a'] for x in treasurechest_s1_r_std])
singlepotofAu_s2= pd.concat([x['linked_a'] for x in treasurechest_s2_r_std])

In [ ]:
singlepotofAu_s1.rename(columns={'TEST_DATE_a':'TEST_DATE'},inplace=True)
singlepotofAu_s2.rename(columns={'TEST_DATE_a':'TEST_DATE'},inplace=True)

Link the annotation sets into single dataframes

In [ ]:
singlepotofAu = pd.concat([singlepotofAu_orig,
                              singlepotofAu_s1,
                              singlepotofAu_s2])
singlepotofAu= singlepotofAu[singlepotofAu['Term_assessed']!='See comment']


In [ ]:
singlepotofAu=singlepotofAu.sort(['Metric','MRN','TEST_CODE','TEST_DATE_month','Term_assessed'], 
                   ascending=[True,True,True,True,False])


In [ ]:
singleAu_MRNonly = singlepotofAu[singlepotofAu.Metric.isin(['CW_bph','CW_narc','CW_feed','CW_psyc','CW_lbp'])]
singleAu_3merge  = singlepotofAu[~singlepotofAu.Metric.isin(['CW_bph','CW_narc','CW_feed','CW_psyc','CW_lbp'])]

singleAu_MRNonly.drop_duplicates(subset=['Metric','MRN','TEST_DATE_month','Term_assessed'], inplace=True)
singleAu_3merge.drop_duplicates(subset=['Metric','MRN','TEST_DATE_month','TEST_CODE','Term_assessed'], inplace=True)

singlepotofAu = pd.concat([singleAu_MRNonly, singleAu_3merge])

____________________________________________________
# At this point,
- (1) each GEM implementation has all numer/not_numer for each metric within single-unified dataframe.
- (2) the annotations are in single dataframe.
____________________________________________________

+ Take the notnumerators from the analyses, that are in [treasurechest numerators, not numerators]
+ See how many of them are marked as such in the treasurechest
+ See how many are marked as not numerators

In [ ]:
# create the directory for exporting results
try:
    os.mkdir('scrapdir/'+coded_date)
except:
    pass

In [ ]:
def eval_results(singleGEMdf, GEM_name, unifiedAu_std, methodx, GEM_or_Au):
    # add tags
    Termcolname_a = 'Term per GEM eval' if GEM_or_Au=='GEM' else 'Term_assessed'
    Termcolname_b = 'Term_assessed' if GEM_or_Au=='GEM' else 'Term per GEM eval'
    if GEM_or_Au =='GEM':
        # (1) split the df into two parts
        used_MRNonly = singleGEMdf[singleGEMdf.Metric.isin(['CW_bph','CW_narc','CW_feed','CW_psyc','CW_lbp'])]
        used_3merge  = singleGEMdf[~singleGEMdf.Metric.isin(['CW_bph','CW_narc','CW_feed','CW_psyc','CW_lbp'])]
        
        # break into not numerator and numerator. 
        used_dMRNonly = used_MRNonly[used_MRNonly[Termcolname_a]=='Not_Numer']
        used_nMRNonly = used_MRNonly[used_MRNonly[Termcolname_a]!='Not_Numer']
        # merge the metric data with annotation data
        used_dMRNonly = used_dMRNonly.merge(unifiedAu_std, on=['Metric', 'MRN', 'TEST_DATE_month'],  how=methodx)
        used_nMRNonly = used_nMRNonly.merge(unifiedAu_std, on=['Metric', 'MRN', 'TEST_CODE', 'TEST_DATE_month'], how=methodx)
        # merge the metric data with annotation data (this works for denominators defined by test)
        used_3merge = used_3merge.merge(unifiedAu_std, on=['Metric', 'MRN', 'TEST_CODE', 'TEST_DATE_month'], how=methodx)
        # bring these three merged df's back into single df.
        used = pd.concat([used_dMRNonly, used_nMRNonly, used_3merge])
        
        
        # drop the rows without Au-standard correlate; 
        #      This means, only assess accuracy for cases identified and reviewed in manual annotations. 
        #      Therefore, cases missed by not being included in denominator were inappropriately lost,
        #      and this error will not be detected by standard measures of sens/specificity/etc
        GEM_asroot=used.dropna(subset=[Termcolname_b])[[GEM_name,'Metric','MRN','TEST_CODE','TEST_DATE_month',Termcolname_a,Termcolname_b]]
        
        # Actually check for the agreement between annotations and automated assignments
        GEM_asroot.loc[GEM_asroot[Termcolname_a]==GEM_asroot[Termcolname_b],'Agree']=1
        GEM_asroot.loc[GEM_asroot[Termcolname_a]!=GEM_asroot[Termcolname_b],'Agree']=0
        ### deleted line: GEM_asroot['Agree']=np.where(GEM_asroot[Termcolname_a]==GEM_asroot[Termcolname_b], 1, 0)
        GEM_asrootNN = GEM_asroot.loc[GEM_asroot[Termcolname_a]=='Not_Numer',:]
        GEM_asrootN = GEM_asroot.loc[GEM_asroot[Termcolname_a]!='Not_Numer',:]
        # determine true and false positive/negatives
        GEM_asrootNN.loc[GEM_asrootNN['Agree']==1,'Agreement_eval']='TNot_Numer'
        GEM_asrootNN.loc[GEM_asrootNN['Agree']!=1,'Agreement_eval']='FNot_Numer'
        ### deleted line: GEM_asrootNN.loc[:,'Agreement_eval']=np.where(GEM_asrootNN['Agree']==1,'TNot_Numer','FNot_Numer')
        GEM_asrootN.loc[GEM_asrootN['Agree']==1,'Agreement_eval']='TNumer'
        GEM_asrootN.loc[GEM_asrootN['Agree']!=1,'Agreement_eval']='FNumer'
        ### deleted line: GEM_asrootN.loc[:,'Agreement_eval']=np.where(GEM_asrootN['Agree']==1,'TNumer','FNumer')
        
        # bring results back together
        GEM_asroot = pd.concat([GEM_asrootNN,GEM_asrootN])
        
        
        intermed = GEM_asroot.groupby(['Metric','Agreement_eval'])['Agree'].count()
        intermed = intermed.reset_index()
        intermed['GEM']= GEM_name
        #idiosyncratic line: splice the string in 'Agreement_eval to capture the u in Numerator
        intermed.loc[:,'GEM result']=intermed['Agreement_eval'].apply(lambda x: '+ Numer' if x[2]=='u' else '- Not Numer')
        intermed.loc[intermed['GEM result']=='+ Numer','Annotated Standard']=intermed['Agreement_eval'].apply(lambda x: '- Not Numer' if x[0]=='F' else '+ Numer')
        intermed.loc[intermed['GEM result']=='- Not Numer','Annotated Standard']=intermed['Agreement_eval'].apply(lambda x: '+ Numer' if x[0]=='F' else '- Not Numer')
        return (GEM_name, GEM_asroot, intermed)
    else:
        print('processing')
        Au_asroot= unifiedAu_std.merge(singleGEMdf, on=['Metric', 'MRN', 'TEST_CODE', 'TEST_DATE_month'], how=methodx)
        Au_asroot.loc[Au_asroot[Termcolname_b].isnull(),'omitted']=1
        intermed = Au_asroot.groupby(['Metric','omitted'])['Metric'].count()
        intermed = intermed.reset_index()
        intermed['GEM']= GEM_name
        return (GEM_name, Au_asroot, )



In [ ]:
#tester = eval_results(allterms_lt[0][1],'_claimto9best', singlepotofAu, 'inner','GEM')
compared = []
for x in allterms_lt:
    compared.append(eval_results(x[1], x[0], singlepotofAu,'inner','GEM'))
unifiedcompared=pd.concat([x[2] for x in compared])

[ For inspecting individual records] 

In [ ]:
jx=allterms_lt[2][1]
jx=jx[jx.MRN=='<insert MRN here>'] # caution --- this is a place where PHI could be inadvertently stored.
jx[['Metric','MRN','TEST_CODE','TEST_DATE','TEST_DATE_month','Term per GEM eval']]

In [ ]:
j=compared[0][1]
j=j[j.Metric=='CW_card']
liv=j[j.Agree==0]
liv

## Back to main method

In [ ]:
tempAu=singlepotofAu
tempAu['bean']=1
tempAu=tempAu.groupby(['Metric','Term_assessed'])['bean'].count()
tempAu=tempAu.reset_index()
tA=tempAu.pivot_table('bean','Metric','Term_assessed')
print('These are the total available annotations within time period')
tA[['Numer', 'Not_Numer']].to_csv('./scrapdir/'+coded_date+'/totalannotationsavail_'+coded_date+'.csv')

In [ ]:
# inspect and export 2x2's for each metric, by GEM (Numerator v. not-numerator assignments)
out=unifiedcompared.pivot_table('Agree', ['Metric','GEM','GEM result'],['Annotated Standard'])
out.to_csv('./scrapdir/'+coded_date+'/totalpivot_'+coded_date+'.csv')


### Calculating sensitivities and specificities

In [ ]:
unifiedcompared=unifiedcompared.fillna(0)
sens_spec = unifiedcompared.pivot_table('Agree', ['Metric','GEM','Annotated Standard'],['GEM result']).fillna(0)
sens_spec.reset_index(inplace=True)
sens_spec.loc[sens_spec['Annotated Standard']=='+ Numer','Sensitivity']=sens_spec['+ Numer'].divide((sens_spec['+ Numer']+ sens_spec['- Not Numer']), axis=0, fill_value=0)#.apply(lambda x: 100*np.round(x,3))
sens_spec.loc[sens_spec['Annotated Standard']=='- Not Numer','Specificity']=sens_spec['- Not Numer'].divide((sens_spec['+ Numer']+ sens_spec['- Not Numer']), axis=0, fill_value=0)#.apply(lambda x: 100*np.round(x,3))


In [ ]:
stem=unifiedcompared.groupby(['Metric','GEM']).count()
stem.reset_index(inplace=True)
stem=stem[['Metric','GEM']]


### Calculating PPV and NPV

In [ ]:
prec_calc = out.fillna(0)
prec_calc.reset_index(inplace=True)

prec_calc.loc[prec_calc['GEM result']=='+ Numer','PPV']=prec_calc['+ Numer'].divide((prec_calc['+ Numer']+ prec_calc['- Not Numer']), axis=0, fill_value=0)#.apply(lambda x: 100*np.round(x,3))
prec_calc.loc[prec_calc['GEM result']=='- Not Numer','NPV']=prec_calc['- Not Numer'].divide((prec_calc['+ Numer']+ prec_calc['- Not Numer']), axis=0, fill_value=0)
prec_calcinter=prec_calc[['Metric','GEM','PPV']].dropna(subset=['PPV']).merge(prec_calc[['Metric','GEM','NPV']].dropna(subset=['NPV']), on=['Metric','GEM'], how='left')
prec_calcinter


### Calculating the Rand Accuracy

In [ ]:
acc_calc = unifiedcompared.pivot_table('Agree', ['Metric','GEM'], 'Agreement_eval')
acc_calc.reset_index(inplace=True)

In [ ]:
acc_calc.fillna(0, inplace=True)

In [ ]:
acc_calc['True Cases']=acc_calc['TNot_Numer']+acc_calc['TNumer']
acc_calc['False Cases']=acc_calc['FNot_Numer']+acc_calc['FNumer']

In [ ]:
acc_calc['Rand Accuracy']=acc_calc['True Cases'].divide((acc_calc['True Cases']+acc_calc['False Cases']), axis=0, fill_value=0)

### Bring Sens, Spec, PPV, NPV, and Rand Accuracy into single DataFrame 
w/ subsequent calculated LR, and LR

In [ ]:
workingassess = stem.merge(sens_spec[['Metric','GEM','Sensitivity']].dropna(subset=['Sensitivity']), on=['Metric','GEM'], how='left')
workingassess = workingassess.merge(sens_spec[['Metric','GEM', 'Specificity']].dropna(subset=['Specificity']), on=['Metric','GEM'], how='left')
workingassess = workingassess.merge(prec_calcinter, on=['Metric','GEM'], how='left')

In [ ]:
workingassess=workingassess.merge(acc_calc[['Metric','GEM','Rand Accuracy','True Cases','False Cases','TNumer','FNumer','TNot_Numer','FNot_Numer']], on=['Metric','GEM'], how='outer')
workingassess=workingassess.fillna(0)
workingassess['LR +']=workingassess['Sensitivity'].divide((1-workingassess['Specificity']), axis=0, fill_value=0)
workingassess['LR -']=(1-workingassess['Sensitivity']).divide((workingassess['Specificity']), axis=0, fill_value=0)
print('PROCESSING UPDATE: \nOutput of Assessment Metrics stored in \'workingassess\' dataframe')

In [ ]:
workingassess.to_csv('./scrapdir/'+coded_date+'/workingassess_'+coded_date+'.csv')